# Getting started with Ibmdbpy - Part 3: Advanced geospatial features 

This notebook is an extensive guide to help you to get familiar with ibmdbpy concepts and geospatial functionalities. It showcases various operations on IdaGeoDataFrames, providing step-by-step examples to get started with the package and its geospatial extension.

___

### Accelerate Python analytics with in-database processing by using ibmdbpy and IBM Db2 Warehouse
 
The ibmdbpy project provides a Python interface for data manipulation and access to in-database algorithms in IBM Db2 Warehouse. It accelerates Python analytics by seamlessly pushing operations written in Python into the underlying database for execution, thereby benefitting from in-database performance-enhancing features, such as columnar storage and parallel processing. For more details about ibmdbpy, please refer to the [documentation](https://pythonhosted.org/ibmdbpy/index.html) and to the dedicated [Git repository](https://github.com/ibmdbanalytics/ibmdbpy/tree/master/ibmdbpy). This notebook provides you with an overview of ibmdbpy geospatial extension.

__About ibmdbpy's geospatial extension__

Ibmdbpy supports a wrapper for spatial functions which enables you to generate and analyze spatial information about geographic features, and to store and manage the data on which this information is based. The spatial data is identified by ibmdbpy as a special class called IdaGeoDataFrame that extends all the properties of an IdaDataFrame and has additional methods supported for geospatial types like ST_Point, ST_LineString, ST_Polygon etc. Like Python package GeoPandas, which is an extension of Pandas and provides the GeoDataFrame abstraction, ibmdbpy spatial extension lets you play with IdaGeoDataFrames and IdaGeoSeries as an extension of IdaDataFrames and IdaSeries. The Python wrappers for spatial functions which Db2 currently supports make the querying process much simpler for users. For more details about ibmdbpy geospatial extension, please refer to the dedicated [documentation](https://pythonhosted.org/ibmdbpy/geospatial.html). More details about Db2 spatial extender can be found on the [IBM Knowledge Center](https://www.ibm.com/support/knowledgecenter/SSCJDQ/com.ibm.swg.im.dashdb.spatial.doc/doc/csbp1001.html).

__Prerequisites__
* Db2 account: see [IBM Cloud](https://cloud.ibm.com/login) or [Db2 Warehouse](https://www.ibm.com/support/knowledgecenter/en/SSCJDQ/com.ibm.swg.im.dashdb.kc.doc/welcome.html)
* Db2 driver: learn more on [IBM Knowledge Center](https://www.ibm.com/support/knowledgecenter/en/SSFMBX/com.ibm.swg.im.dashdb.doc/connecting/connect_applications_by_type.html) and see [IBM Support](https://www.ibm.com/support/pages/db2-jdbc-driver-versions-and-downloads)
* Having installed the [ibmdbpy](https://pypi.org/project/ibmdbpy/) python library with pip: 
> pip install ibmdbpy 
* Optional dependency for JDBC is the [jaydebeapi](https://pypi.org/project/JayDeBeApi/) library. Run the following command to install ibmdbpy, as well as the dependencies for the JDBC feature:
> pip install ibmdbpy[jdbc]

__Contents__ 

1. __Establish connection to Db2 database__
        1. JDBC connection
        2. Verbosity and autocommit settings
2. __Create and open IdaGeoDataFrames__
        1. Load sample data from Db2 tables
        2. Extract an IdaGeoSeries
        3. Create sample tables with various geometry types    
3. __Get familiar with geospatial operations__ 
        1. General properties of geometry columns
        2. Dealing with point coordinates
        3. Start, middle and end points of curves
        4. Area and boundaries of polygons
        5. Length versus perimeter
        6. Envelope, MBR and convex hull
        7. Centroid and buffer 
        8. Rings
        9. Generalization of a geospatial pattern 
        10. Boolean methods
        11. Operations on multiple geometry columns

__Imports__

In [1]:
from ibmdbpy import IdaDataBase, IdaDataFrame, IdaGeoDataFrame

## 1. Establish connection to Db2 database

In ibmdbpy, users can choose to use JDBC to connect to a remote Db2 instance. The JDBC Connection is based on a Java Virtual Machine, so it is available on every machine that can run Java. You could also use an ODBC connection, however this is not the option we use in this notebook.

__1. JDBC connection__

First, you need to dowload a valid driver (more info on [IBM Support](https://www.ibm.com/support/pages/db2-jdbc-driver-versions-and-downloads)). Then you need to put the `db2jcc.jar` or`db2jcc4.jar` file in the ibmdbpy site-package folder. When ibmdbpy runs, it checks whether one of those files exists in its installation folder and uses it to establish the connection. 

More details on IBM Knowledge Center:
* JDBC for [IBM Db2 Warehouse](https://www.ibm.com/support/knowledgecenter/en/SSCJDQ/com.ibm.swg.im.dashdb.doc/connecting/connect_connecting_jdbc_applications.html)
* JDBC for [IBM Db2 on Cloud](https://www.ibm.com/support/knowledgecenter/en/SSFMBX/com.ibm.swg.im.dashdb.doc/connecting/connect_connecting_jdbc_applications.html)

In [2]:
#Enter the values for you database connection
dsn_database = "___" # e.g. "BLUDB"
dsn_hostname = "___" # e.g.: "abc.url.example"
dsn_port = "___"    # e.g. "50000"
dsn_uid = "___"     # e.g. "db2_1234"
dsn_pwd = "___"     # e.g. "zorglub"

In [3]:
connection_string='jdbc:db2://'+dsn_hostname+':'+dsn_port+'/'+dsn_database+':user='+dsn_uid+';password='+dsn_pwd+";" 
# the IdaDataBase object holds the connection to database.
idadb=IdaDataBase(dsn=connection_string)

Congratulations! You successfully connected to Db2 with ibmdbpy! When you are done, use `idadb.close()` to close the connection. To reconnect, or if the connection was broken, just use `idadb.reconnect()`. 

__2. Verbosity and autocommit settings__

The verbose mode automatically prints all SQL-communication between ibmdbpy and Db2, which can be very useful for debugging or understanding how ibmdbpy works. Choose the mode with `set_verbose()` or by setting the `verbosity` option when defining the IdaDataBase object. We encourage you to take a look at the prints in the first place, then feel free to silence the verbose. Note that printing adds delay when running cells.  

In [4]:
# Verbosity
from ibmdbpy.utils import set_verbose
# if you want to see what takes place under the hood : set verbose to True
set_verbose(False) # set it to True if you want to see the detail of ibmdbpy's operations

By default the environment variable `AUTOCOMMIT` is then set to True, which means that every SQL statement which is submitted through the connection is executed within its own transaction and then committed implicitly. When you close the connection to Db2, if the environment variable `AUTOCOMMIT` is set to False, then all changes after the last explicit commit are discarded. 

Let's get to it!

## 2. Create and open IdaGeoDataFrames

Let's explore the functionalities of ibmdbpy's geospatial extension! In this notebook, we will get familiar with IdaGeoDataFrames. An IdaGeoDataFrame is a reference to a spatial table in a remote instance of Db2. It has inherited the properties of an IdaDataFrame and bebefits from additional functionalities derived from Db2 spatial extension. 

In this notebook, we will use sample data available out of the box in Db2 Warehouse. The `SAMPLES.GEO_COUNTY` dataset contains geographical and administrative data about US counties. The `SAMPLES.GEO_TORNADO` dataset holds 5 decades of tornado records in the US. We will also use much smaller sample table of various data types. We will create these tables ourselves with a few SQL statements.

__1. Load sample data from Db2 tables__

Here we show you how to directly open Db2 tables as IdaGeoDataFrames.

When defining an IdaGeoDataFrame, the first argument is the name of the IdaDataBase object which holds the connection to the database, the second argument designates the Db2 table you want to open. Optionally you may set an indexer column and a geometry column. Indexer and geometry can be defined and / or changed anytime using `indexer` and `set_geometry`.

Note that in the following cell we have set the `OBJECTID` column as indexer when defining the IdaDataFrame. Otherwise, since the data is partitioned, it cannot be guaranteed that rows are always displayed in the same order. (Although, in paractice, an implicit sorting is operated by ibmdbpy). To ensure a behavior closer to Pandas' we therefore explicitly set an eligible column as index. You can either directly set the geometry column when defining an IdaGeoDataFrame object, or set it afterwards with `set_geometry('<column_name>')`.

In [5]:
# prepopulated table in Db2

counties = IdaGeoDataFrame(idadb,'SAMPLES.GEO_COUNTY',indexer='OBJECTID')
counties.set_geometry('SHAPE')

tornadoes = IdaGeoDataFrame(idadb,'SAMPLES.GEO_TORNADO',indexer='OBJECTID')
tornadoes.set_geometry('SHAPE')

/Users/eva.feillet/.local/lib/python3.7/site-packages/jaydebeapi/__init__.py:250: UserWarning: No type mapping for JDBC type 'STRUCT' (constant value 2002). Using None as a default type_code.
  "Using None as a default type_code." % (type_name, jdbc_type_const))


__2. Extract an IdaGeoSeries__

An IdaGeoSeries inherits the properties of IdaSeries. The specificity of IdaGeoSeries relies in the data type: `ST_GEOMETRY` or one of its subtypes: `ST_POINT`, `ST_LINESTRING`, `ST_MULTIPOLYGON` etc. Through this notebook, you will get familiar with them.

To isolate an IdaGeoSeries, just select the corresponding column of the IdaGeoDataFrame, like you would do in Pandas.

In [6]:
idageoseries = counties["SHAPE"]

Let's take a look at the data type:

In [7]:
idageoseries.dtypes

,TYPENAME
SHAPE,ST_MULTIPOLYGON


Each element contains a list of list of coordinates. Coordinates define points. Each list of coordinates defines a polygon. A set of polygons makes a multipolygon. Same applies for curves (called linestrings and multilinestrings).

In [8]:
idageoseries.head()

0    MULTIPOLYGON (((-119.0414320216 46.1928618799,...
1    MULTIPOLYGON (((-82.3369660445 40.5550088866, ...
2    MULTIPOLYGON (((-82.6497086900 33.6087744671, ...
3    MULTIPOLYGON (((-84.1141995028 39.5779906201, ...
4    MULTIPOLYGON (((-94.1405779888 35.0996842940, ...
Name: SHAPE, dtype: object

__3. Create sample tables with various geometry types__

A separate text file called `sql_script` containing SQL statements accompanies this notebook. You can either copy these statements and let them run directly in your Db2 Warehouse console, or simply run the next cell of this notebook to execute the queries automatically with ibmdbpy. 

You obtain a few sample tables with toy data. They will be useful to showcase some particular geospational functionalities of ibmdbpy.

In [9]:
with open("sql_script", "r") as f:
    for line in f:
        print(line)
        idadb.ida_query(str(line))

DROP TABLE sample_polygons;

SAMPLE_POLYGONS
Table already exists.
DROP TABLE sample_geometries; 

SAMPLE_GEOMETRIES
Table already exists.
DROP TABLE sample_points;

SAMPLE_POINTS
Table already exists.
DROP TABLE sample_mlines;

SAMPLE_MLINES
Table already exists.
DROP TABLE sample_lines;

SAMPLE_LINES
Table already exists.
SET CURRENT FUNCTION PATH = CURRENT FUNCTION PATH, db2gse;

CREATE TABLE sample_polygons (id INTEGER, geometry db2gse.ST_Polygon) ORGANIZE BY ROW;

INSERT INTO sample_polygons VALUES (1101, db2gse.ST_Polygon ('polygon ((110 120, 110 140, 120 130, 110 120))', 1));

INSERT INTO sample_polygons VALUES (1102, db2gse.ST_Polygon ('polygon ((110 120, 110 140, 130 140, 130 120, 110 120), (115 125, 115 135, 125 135, 125 135, 115 125))', 1));                    

CREATE TABLE sample_geometries (id INTEGER, geometry db2gse.ST_Geometry) organize by row;

INSERT INTO sample_geometries(id, geometry) VALUES (1, db2gse.ST_Geometry('point(1 2)', 1) );

INSERT INTO sample_geometries(

In [10]:
# Define the corresponding IdaDataFrames

sample_points = IdaGeoDataFrame(idadb, "SAMPLE_POINTS", indexer = "ID", geometry = "LOC")
sample_lines = IdaGeoDataFrame(idadb, "SAMPLE_LINES", indexer = "ID", geometry = "GEOMETRY")
sample_mlines = IdaGeoDataFrame(idadb, "SAMPLE_MLINES", indexer = "ID", geometry = "GEOMETRY")
sample_geometries = IdaGeoDataFrame(idadb, "SAMPLE_GEOMETRIES", indexer = "ID", geometry = "GEOMETRY")
sample_polygons = IdaGeoDataFrame(idadb, "SAMPLE_POLYGONS", indexer = "ID", geometry = "GEOMETRY")

Great! We now have everything we need to explore ibmdbpy's geospatial features!

## 3. Get familiar with geospatial operations 

In the following cells of this notebook, you will find examples for each of the geospatial methods ot the package which return an IdaGeoSeries. These methods use functions from Db2 geospatial extender (db2gse). All you need is the sample data we defined above.

__1. General properties of geometry columns__

<font color='blue'>Note on geospatial methods</font>

When calling methods to perform operations on IdaGeoDataFrames, these operations will automatically be performed on the chosen geometry column(s). 

For example, if you want to compute the area of a county, *provided that the column containing the coordinates of each county as a collection of polygons has been set as geometry column*, calling `counties.area()` will compute the area of each county. 
> counties["AREA"] = counties.area()

A new column called `AREA` is created in the Db2 table. The area is computed on the basis of the collection of polygons contained in the `SHAPE` column of the IdaGeoDataFrame, which we have defined as the geometry column.

But if you want to perform an operation on a geospatial column which is not defined as *the* geometry column, then you may either set this column as the new geometry column, or explicitly specify this column as the targeted IdaGeoSeries of the method you call. 
> counties["AREA"] = counties["SHAPE"].area()

Note that all of the methods we will show you in this notebook return null if the input geometry is null or empty, and will raise an error if the input geometry has not the appropriate data type. If not specified otherwise, the returned geometries have the same spatial reference system as the input geometry.

* Setting a geometry column as *the* geometry column

Db2 documentation might refer to columns with geospatial data types as __geometries__. But don't get confused about it: there is __only one column at a time__ which can be defined as __*the* geometry__ column of a dataset. 

Here we have directly defined the geometry column when defining the IdaDataFrame:
> sample_points = IdaGeoDataFrame(idadb, "SAMPLE_POINTS", indexer = "ID", geometry = "LOC")

If you want to set another column as geometry, use this structure:
> your_idageodf.set_geometry("eligible_column")

You will find useful to check which column is defined as your current "official" geometry column by calling the attributes `geometry.columns`. 

In [11]:
print(sample_geometries.geometry.columns)
print(tornadoes.geometry.columns)
print(sample_points.geometry.columns)

['GEOMETRY']
['SHAPE']
['LOC']


* Data type and Geometry type

Each geospatial method of ibmdbpy may have restrictions as to which data type is allowed as input. You can check the data type(s) of any geometry column by using the `geometry_type` method and `dtypes` attribute. What is the difference? `geometry_type` gives the particular geometry type of each row, whereas `dtypes` gives you the data type by column.

Geometry columns can contain elements of different data types! For example, a column containing LineStrings and MultiLineStrings will have `ST_MultiLineString` as overall data type. A column containing points and lines or polygons will have the general `ST_GEOMETRY` data type.


The spatial data types supported by DB2 Spatial Extender areimplementations of the geometries shown in the figure.

![](geom_dtypes.png)

*Fig: Hierarchy of geometries supported by DB2 Spatial Extender. Instantiable geometries inthis figure include examples of how they might be rendered visually.*

In [12]:
# Geometry type --> output by row
sample_geometries['GEOMETRY'].geometry_type().head()

0           "DB2GSE  "."ST_POINT"
1         "DB2GSE  "."ST_POLYGON"
2           "DB2GSE  "."ST_POINT"
3    "DB2GSE  "."ST_MULTIPOLYGON"
4      "DB2GSE  "."ST_LINESTRING"
Name: DB2GSE.ST_GEOMETRYTYPE(GEOMETRY), dtype: object

In [13]:
# dtypes --> output by column
sample_geometries[['GEOMETRY']].dtypes

,TYPENAME
GEOMETRY,ST_GEOMETRY


* Dimension of geometries

If the given geometry is empty, then -1 is returned. For points and multipoints, the dimension is 0; for curves and multicurves, the dimension is 1; and for polygons and multipolygons, the dimension is 2.  If the given geometry is null, then null is returned. Here are various examples of the results you can obtain.

In [14]:
# Mixed types and Empty geometries
sample_geometries["DIM"] = sample_geometries.dimension()
sample_geometries[["GEOMETRY", "DIM"]].head()

,GEOMETRY,DIM
0,POINT (1.000000 2.000000),0
1,"POLYGON ((0.000000 0.000000, 5.000000 0.000000...",2
2,POINT EMPTY,-1
3,MULTIPOLYGON EMPTY,-1
4,"LINESTRING (33.000000 2.000000, 34.000000 3.00...",1


In [15]:
# Dimension of a MultiLineString
tornadoes["SHAPE_DIM"] = tornadoes.dimension()
tornadoes[["SHAPE", "SHAPE_DIM"]].head()

,SHAPE,SHAPE_DIM
0,MULTILINESTRING ((-90.2200062071 38.7700071663...,1
1,MULTILINESTRING ((-89.3000059755 39.1000072739...,1
2,MULTILINESTRING ((-84.5800047496 40.8800078382...,1
3,MULTILINESTRING ((-94.3700070010 34.4000061520...,1
4,MULTILINESTRING ((-90.6800062393 37.6000069289...,1


In [16]:
# Dimension of a polygon

tornadoes["buffer_20_km"] = tornadoes.buffer(distance = 20, unit = "KILOMETER")
# Note: see section 7. to learn more about the buffer method

tornadoes["buffer_20_km_dim"] = tornadoes["buffer_20_km"].dimension()
tornadoes[["buffer_20_km", "buffer_20_km_dim"]].head()

,buffer_20_km,buffer_20_km_dim
0,"POLYGON ((-90.3065519651 38.9369737029, -90.32...",2
1,"POLYGON ((-89.3798853739 39.2690904737, -89.39...",2
2,"POLYGON ((-84.7257488606 41.0222185578, -84.73...",2
3,"POLYGON ((-94.5212609425 34.5296645617, -94.53...",2
4,"POLYGON ((-90.8575378881 37.7120296620, -90.86...",2


* Number of items in a geometry

You can compute the number of geometries in a collection with `num_geometries`. Similarly, to know how many points, lines or polygons a given geometry contains, use `num_points`, `num_line_strings` and `num_polygons`.

In [17]:
# Num Geometries for an ST_MULTIPOLYGON column
print(counties.geometry.dtypes)
counties.num_geometries().head()

              TYPENAME
SHAPE  ST_MULTIPOLYGON


0    1
1    1
2    1
3    1
4    1
Name: DB2GSE.ST_NUMGEOMETRIES(SHAPE), dtype: int64

In [18]:
# Num Geometries for an ST_MULTILINESTRING column
print(sample_mlines.geometry.dtypes)
sample_mlines.num_geometries().head()

                    TYPENAME
GEOMETRY  ST_MULTILINESTRING


0    3
Name: DB2GSE.ST_NUMGEOMETRIES(GEOMETRY), dtype: int64

You can be more precise and get the number of geometries of a particular type: points, lines and polygons.

In [19]:
# Num Points
sample_geometries["num_points"] = sample_geometries.num_points()
sample_geometries[["GEOMETRY", "num_points"]].head()

,GEOMETRY,num_points
0,POINT (1.000000 2.000000),1.0
1,"POLYGON ((0.000000 0.000000, 5.000000 0.000000...",5.0
2,POINT EMPTY,NaN
3,MULTIPOLYGON EMPTY,NaN
4,"LINESTRING (33.000000 2.000000, 34.000000 3.00...",3.0


In [20]:
# Num Linestrings
sample_mlines.num_line_strings().head()

0    3
Name: DB2GSE.ST_NUMLINESTRINGS(GEOMETRY), dtype: int64

In [21]:
# Num Polygons
counties['NUM_POLY'] = counties.num_polygons()
print(counties['NUM_POLY'][counties['NUM_POLY']>1].shape)
counties['NUM_POLY'][counties['NUM_POLY']>1].head()

(57, 1)


0    2
1    2
2    2
3    2
4    2
Name: NUM_POLY, dtype: int64

__2. Dealing with point coordinates__

When you have a geometry column containing points or a list of points, you might need to extract some of the coordinates or compute min and max in a particular dimension. Let's see how it works.

* Dimension of point coordinates

Use this method to check the dimension of your coordinates. Here you see that the points obtained with the `centroid` method have 2 dimensions (e.g. X and Y), whereas some of the sample points we have created have a Z and/or an M coordinate. 

X, Y and Z are traditional cartesian coordinates. The measure value M can be used to record additional non-coordinte values at various points within a geometry, for example an angle or a temperature.

In [22]:
# Sample points
sample_points['coord_dim'] = sample_points.coord_dim()
sample_points[['ID', 'LOC','coord_dim']].head()

,ID,LOC,coord_dim
0,1,POINT (14.000000 58.000000),2
1,2,POINT Z(12.000000 35.000000 12),3
2,3,POINT ZM(12.000000 66.000000 43 45),4
3,4,POINT M(14.000000 58.000000 4),3
4,5,POINT Z(12.000000 35.000000 12),3


* Extract coordinates

In [23]:
# X, Y, Z and M coordinates

sample_points_extractor = IdaGeoDataFrame(idadb, "SAMPLE_POINTS", indexer = "ID")
sample_points_extractor.set_geometry("LOC")

sample_points_extractor["X"] = sample_points_extractor.x()
sample_points_extractor["Y"] = sample_points_extractor.y()
sample_points_extractor["Z"] = sample_points_extractor.z()
sample_points_extractor["M"] = sample_points_extractor.m()
sample_points_extractor.head()

,ID,LOC,X,Y,Z,M
0,1,POINT (14.000000 58.000000),14.0,58.0,NaN,NaN
1,2,POINT Z(12.000000 35.000000 12),12.0,35.0,12.0,NaN
2,3,POINT ZM(12.000000 66.000000 43 45),12.0,66.0,43.0,45.0
3,4,POINT M(14.000000 58.000000 4),14.0,58.0,NaN,4.0
4,5,POINT Z(12.000000 35.000000 12),12.0,35.0,12.0,NaN


* Min and max

In [24]:
# Max X, Y, Z and M
sample_geometries = IdaGeoDataFrame(idadb, "SAMPLE_GEOMETRIES", indexer = "ID", geometry = "GEOMETRY")

sample_geometries["max_X"] = sample_geometries.max_x()
sample_geometries["max_Y"] = sample_geometries.max_y()
sample_geometries["max_Z"] = sample_geometries.max_z()
sample_geometries["max_M"] = sample_geometries.max_m()
sample_geometries.head()

,ID,GEOMETRY,max_X,max_Y,max_Z,max_M
0,1,POINT (1.000000 2.000000),1.0,2.0,None,None
1,2,"POLYGON ((0.000000 0.000000, 5.000000 0.000000...",5.0,4.0,None,None
2,3,POINT EMPTY,NaN,NaN,None,None
3,4,MULTIPOLYGON EMPTY,NaN,NaN,None,None
4,5,"LINESTRING (33.000000 2.000000, 34.000000 3.00...",35.0,6.0,None,None


In [25]:
# Min X, Y, Z and M
sample_geometries = IdaGeoDataFrame(idadb, "SAMPLE_GEOMETRIES", indexer = "ID", geometry = "GEOMETRY")

sample_geometries["min_X"] = sample_geometries.min_x()
sample_geometries["min_Y"] = sample_geometries.min_y()
sample_geometries["min_Z"] = sample_geometries.min_z()
sample_geometries["min_M"] = sample_geometries.min_m()
sample_geometries.head()

,ID,GEOMETRY,min_X,min_Y,min_Z,min_M
0,1,POINT (1.000000 2.000000),1.0,2.0,None,None
1,2,"POLYGON ((0.000000 0.000000, 5.000000 0.000000...",0.0,0.0,None,None
2,3,POINT EMPTY,NaN,NaN,None,None
3,4,MULTIPOLYGON EMPTY,NaN,NaN,None,None
4,5,"LINESTRING (33.000000 2.000000, 34.000000 3.00...",33.0,2.0,None,None


__3. Start, middle and end point of curves__

These methods take a curve (`ST_LineString`) as input and return the corresponding points:
* start point: the first point of the curve
* last point: the last point of the curve
* mid point: the point geometrically in the middle of the two points above. 

In [26]:
# Start point
sample_lines['start'] = sample_lines.start_point()
# Mid point
sample_lines["middle"] = sample_lines.mid_point()
# End point
sample_lines['end'] = sample_lines.end_point()

# Take a look at the new columns
sample_lines.head()

,ID,GEOMETRY,start,middle,end
0,1110,"LINESTRING (850.000000 250.000000, 850.000000 ...",POINT (850.000000 250.000000),POINT (850.000000 550.000000),POINT (850.000000 850.000000)
1,1111,"LINESTRING (90.000000 90.000000, 100.000000 10...",POINT (90.000000 90.000000),POINT (95.000000 95.000000),POINT (100.000000 100.000000)


__4. Area and boundaries of polygons__

You can compute the area and boundary of geometries with type `ST_Polygon` or `ST_MultiPolygon`. Let's see what we obtain.

* Area of a surface

If the input surface is composed of several polygons, then the area is the sum of each polygon's areas.

In [27]:
# Area
counties['area_in_km2'] = counties.area(unit = 'KILOMETER')
counties[['NAME','area_in_km2']].head()

,NAME,area_in_km2
0,Hall,1430.061292
1,Baker,898.279987
2,Wright,1508.045410
3,Charlotte,1243.314514
4,Sangamon,2280.381965


Under the hood, an SQL statements using `db2gse.ST_AREA` function is applyed to the `SHAPE`column, defined as geometry column. You can print the corresponding statements by enabling the `verbose` option.

<font color="blue">Note on allowed units</font>

To see which linear units are allowed, use the `linear_units` method of IdaGeoDataFrames. Examples include FOOT, MILE, METER, KILOMETRE, INDIAN YARD, NAUTICAL MILE and many other country and time specific units.

* Boundary of a shape

The `boundary` method returns the boundary of the input geometry as a new geometry column. Here you can see that the boundary of a polygon with no holes is a single linestring, represented as ST_LineString (example: Hodgeman county). The boundary of a polygon with one or more holes would consist of multiple linestrings, represented as ST_MultiLineString (example: Bedford county). 

In [28]:
# Boundary
counties['boundary'] = counties.boundary()
counties[['NAME','boundary']].head()

,NAME,boundary
0,Wilbarger,"LINESTRING (-99.4756582604 33.8340108094, -99...."
1,Austin,"LINESTRING (-96.6219873342 30.0442882117, -96...."
2,Logan,"LINESTRING (-99.4497297204 46.6316377481, -99...."
3,La Plata,"LINESTRING (-107.4817473750 37.0000108736, -10..."
4,Randolph,"LINESTRING (-91.2589262966 36.2578866492, -91...."


<font color="blue">Note on Spatial Reference Systems (SRS)</font>

The resulting coordinates are given in the same spatial reference system as the input coordinates. This is also true for the other methods showcased in this notebook. Let's check this with the `srid` method. If no spatial reference system identifier is given as an input parameter, the `srid` method simply returns the current spatial reference system identifier of the given geometry. Otherwise it resets the ID of the spatial reference system.

In [29]:
# Look at the reference system
# The bounadry column is not the specified geometry column, 
# so we explicitly call the srid function on it.
counties['boundary'].srid().head(3)

0    1005
1    1005
2    1005
Name: DB2GSE.ST_SRID(DB2GSE.ST_BOUNDARY(SHAPE)), dtype: int64

In [30]:
# Here the srid function is applied to the SHAPE column
# because it is set as the geometry column of the IdaGeoDataFrame.
counties.srid().head(3)

0    1005
1    1005
2    1005
Name: DB2GSE.ST_SRID(SHAPE), dtype: int64

Note that you can additionally check the *name* of the spatial reference system of your data with `srs_name`.

In [31]:
counties.srs_name().head()

0    SAMPLE_GCS_WGS_1984
1    SAMPLE_GCS_WGS_1984
2    SAMPLE_GCS_WGS_1984
3    SAMPLE_GCS_WGS_1984
4    SAMPLE_GCS_WGS_1984
Name: DB2GSE.ST_SRSNAME(SHAPE), dtype: object

__5. Length versus perimeter__

The `length` method is for curves or collection of curves, i.e. geometries with data type `ST_LineString` or `ST_MultiLineString`. The`perimeter` method is a method for surfaces or collection of surfaces, i.e. geometries with data type ST_Polygon or ST_MultiPolygon. 

In [32]:
# Length
tornadoes['LEN_KM'] = tornadoes.length(unit = 'KILOMETER')
tornadoes[['OBJECTID', 'SHAPE', 'LEN_KM']].head()

,OBJECTID,SHAPE,LEN_KM
0,1,MULTILINESTRING ((-90.2200062071 38.7700071663...,17.798545
1,2,MULTILINESTRING ((-89.3000059755 39.1000072739...,6.448745
2,3,MULTILINESTRING ((-84.5800047496 40.8800078382...,0.014213
3,4,MULTILINESTRING ((-94.3700070010 34.4000061520...,0.014173
4,5,MULTILINESTRING ((-90.6800062393 37.6000069289...,4.254681


In [33]:
# Perimeter
counties["PERI_MILE"] = counties.perimeter(unit = 'MILE')
counties[["NAME", "SHAPE", "PERI_MILE"]].head()

,NAME,SHAPE,PERI_MILE
0,Crowley,"MULTIPOLYGON (((-104.0582517183 38.1464982543,...",116.393352
1,Crook,"MULTIPOLYGON (((-119.6555997519 44.3069696862,...",261.563538
2,Somerset,"MULTIPOLYGON (((-75.9484559154 38.2159676862, ...",126.374737
3,Clarke,"MULTIPOLYGON (((-77.9619585898 39.0138487739, ...",55.450347
4,Erie,"MULTIPOLYGON (((-78.4643840244 42.8674694862, ...",173.074296


__6. Envelope, MBR and Convex hull__

What is the difference between the envelope, the Minimum Bounding Rectangle and the convex hull of a geometry?

All 3 methods return a shape containing the input geometry, but these shapes do not have the same properties.

* The Minimum Bounding Rectangle (MBR) of a geometry is the smallest rectangle to contain the whole input shape. You can obtain it with the `mbr` method.
* The `envelope` method outputs simple rectangles, represented as a polygon. If the given geometry is a point, a horizontal linestring, or a vertical linestring, then a rectangle, which is slightly larger than the given geometry, is returned. Otherwise, the Minimum Bounding Rectangle of the geometry is returned as the envelope.
* The `convex hull` method returns the smallest *convex* set that contains the input shape. The convex hull is also called convex envelope or convex closure. For example, if you have a bounded subset of points in the Euclidean space, the convex hull can be visualized as the shape enclosed by an elastic band stretched around the outside points of the subset. 

Note that if vertices of the geometry do not form a convex, the  convexhull method returns a null.
If possible, the specific type of the returned geometry will be `ST_Point`, `ST_LineString`, or `ST_Polygon`. The convex hull of a convex polygon with no holes is a single linestring, represented as ST_LineString. The convex hull of a non convex polygon does not exit. For None geometries, for empty geometries and for non convex geometries the output is None.

In [34]:
# MBR
counties["MBR"] = counties.mbr()
# Envelope
counties['envelope'] = counties.envelope()
# Convex Hull
counties['convex_envelope'] = counties.convex_hull()

# Compare coordinates
counties[["NAME", "SHAPE", "MBR", 'envelope', 'convex_envelope']].head()

,NAME,SHAPE,MBR,envelope,convex_envelope
0,Lafayette,"MULTIPOLYGON (((-90.4263836312 42.5071807967, ...","POLYGON ((-90.4269086653 42.5056648248, -89.83...","POLYGON ((-90.4269086653 42.5056648248, -89.83...","POLYGON ((-90.4269086653 42.8128698462, -90.42..."
1,Sanilac,"MULTIPOLYGON (((-82.1455052616 43.6955954588, ...","POLYGON ((-83.1204005291 43.1541073218, -82.12...","POLYGON ((-83.1204005291 43.1541073218, -82.12...","POLYGON ((-83.1204005291 43.3270573447, -83.11..."
2,Taylor,"MULTIPOLYGON (((-84.0691810519 32.5918031946, ...","POLYGON ((-84.4532361602 32.3720591397, -84.00...","POLYGON ((-84.4532361602 32.3720591397, -84.00...","POLYGON ((-84.4532361602 32.5423801709, -84.45..."
3,Ohio,"MULTIPOLYGON (((-80.5191234475 40.0164178652, ...","POLYGON ((-80.7338065145 40.0164178652, -80.51...","POLYGON ((-80.7338065145 40.0164178652, -80.51...","POLYGON ((-80.7338065145 40.0334168595, -80.59..."
4,Houston,"MULTIPOLYGON (((-83.7877562454 32.5016909466, ...","POLYGON ((-83.8568549803 32.2825891390, -83.48...","POLYGON ((-83.8568549803 32.2825891390, -83.48...","POLYGON ((-83.8568549803 32.3751601541, -83.85..."


__7. Buffer and Centroid__

The `buffer` method returns the geometry that surrounds the input geometry by the input distance, measured in a given unit. 

Note that any circular curve in the boundary of the resulting geometry is approximated by linear strings! For example, the buffer around a point, which would result in a circular region, is approximated by a polygon whose boundary is a linestring.

In [35]:
# Buffer
tornadoes["buffer_20_km"] = tornadoes.buffer(distance = 20, unit = "KILOMETER")
tornadoes[['SHAPE', 'buffer_20_km']].head()

,SHAPE,buffer_20_km
0,MULTILINESTRING ((-90.2200062071 38.7700071663...,"POLYGON ((-90.3065519651 38.9369737029, -90.32..."
1,MULTILINESTRING ((-89.3000059755 39.1000072739...,"POLYGON ((-89.3798853739 39.2690904737, -89.39..."
2,MULTILINESTRING ((-84.5800047496 40.8800078382...,"POLYGON ((-84.7257488606 41.0222185578, -84.73..."
3,MULTILINESTRING ((-94.3700070010 34.4000061520...,"POLYGON ((-94.5212609425 34.5296645617, -94.53..."
4,MULTILINESTRING ((-90.6800062393 37.6000069289...,"POLYGON ((-90.8575378881 37.7120296620, -90.86..."


The other way around: The `centroid` method returns the geometric center of the input geometry. This point is defined as the center of the minimum bounding rectangle of the given geometry. The resulting point is represented in the spatial reference system of the given geometry.

In [36]:
# Centroid
counties['centroid'] = counties.centroid()
counties[['NAME','centroid']].head()

,NAME,centroid
0,Lafayette,POINT (-90.1321430727 42.6602443432)
1,Sanilac,POINT (-82.6226543867 43.4248513903)
2,Taylor,POINT (-84.2276055947 32.5598456804)
3,Ohio,POINT (-80.6263529839 40.1009943786)
4,Houston,POINT (-83.6721899279 32.4873821842)


__8. Rings__

You can create a ring around a surface, or count the rings inside a geometry. More precisely, `num_interior_ring` gives the number of rings inside a polygon and `exterior_ring` computes the coordinates of the exterior ring of a polygon. 

In [37]:
# Exterior ring
sample_polygons["ext_ring"] = sample_polygons.exterior_ring()
sample_polygons.head()

,ID,GEOMETRY,ext_ring
0,1101,"POLYGON ((110.000000 120.000000, 120.000000 13...","LINESTRING (110.000000 120.000000, 120.000000 ..."
1,1102,"POLYGON ((110.000000 120.000000, 130.000000 12...","LINESTRING (110.000000 120.000000, 130.000000 ..."


In [38]:
# num Interior Ring
sample_polygons["int_ring"] = sample_polygons.num_interior_ring()
sample_polygons[["GEOMETRY", "int_ring"]].head()

,GEOMETRY,int_ring
0,"POLYGON ((110.000000 120.000000, 120.000000 13...",0
1,"POLYGON ((110.000000 120.000000, 130.000000 12...",1


__9. Generalization of geospatial patterns__

The `generalize` takes a threshold value as option and represents the given geometry with a reduced number of points, while preserving the general characteristics of this geometry.

It uses the Douglas-Peucker line-simplification algorithm. This algorithm recursively subdivides the sequence of points which define the geometry, until a set of points can be replaced by a straight line segment. In this line segment, none of the defining points deviates from the straight line segment by more than the given threshold. For simplification, Z and M coordinates are not considered when performing this operation. 

In [39]:
# Generalize
tornadoes['generalize'] = tornadoes.generalize(threshold = 4)
tornadoes[['OBJECTID', 'SHAPE', 'generalize']].head()

,OBJECTID,SHAPE,generalize
0,1,MULTILINESTRING ((-90.2200062071 38.7700071663...,MULTILINESTRING ((-90.2200062071 38.7700071663...
1,2,MULTILINESTRING ((-89.3000059755 39.1000072739...,MULTILINESTRING ((-89.3000059755 39.1000072739...
2,3,MULTILINESTRING ((-84.5800047496 40.8800078382...,MULTILINESTRING ((-84.5800047496 40.8800078382...
3,4,MULTILINESTRING ((-94.3700070010 34.4000061520...,MULTILINESTRING ((-94.3700070010 34.4000061520...
4,5,MULTILINESTRING ((-90.6800062393 37.6000069289...,MULTILINESTRING ((-90.6800062393 37.6000069289...


__10. Boolean methods__

Last but not least in our toolbox: a few convenience methods for property checking.

* General

The method `is_valid`tells you whether an element is topologically correct.

In [40]:
# is_valid
sample_points.is_valid().head()
# Hopefully, we obtain only 1s (True)

0    1
1    1
2    1
3    1
4    1
Name: DB2GSE.ST_ISVALID(LOC), dtype: int64

* Methods for points

Use `is_3d` to assess if a point is 3-dimensional, and `is_measured` to know if it has an additional measure property.

In [41]:
# is_3d
sample_points["is_3d"] = sample_points.is_3d()

# is_measured
sample_points["is_M"]=sample_points.is_measured()
sample_points.head()

,ID,LOC,coord_dim,is_3d,is_M
0,1,POINT (14.000000 58.000000),2,0,0
1,2,POINT Z(12.000000 35.000000 12),3,1,0
2,3,POINT ZM(12.000000 66.000000 43 45),4,1,1
3,4,POINT M(14.000000 58.000000 4),3,0,1
4,5,POINT Z(12.000000 35.000000 12),3,1,0


* Geometric properties of shapes

Quite intuitively, a geometry is said to be empty if it does not have any points.

In [42]:
#is_empty
counties["boundary"].is_empty().head(3)

0    0
1    0
2    0
Name: DB2GSE.ST_ISEMPTY(DB2GSE.ST_BOUNDARY(SHAPE)), dtype: int64

Objects with type `ST_LineStrings`, `ST_MultiPoints`, and `ST_MultiLineStrings` are either simple or nonsimple. They are simple if they obey all topological rules that are imposed on their respective subclass and nonsimple if they bend a few rules. Here are the rules: `ST_LineString` is simple if it does not intersect its interior.
`ST_MultiPoint` is simple if none of its elements occupy the same coordinate space. `ST_MultiLineString` is simple if none of its element's interiors intersect.

In [43]:
# is_simple
counties["is_simple"] = counties.is_simple()
filtered_counties = counties[counties['is_simple'] == 0]
filtered_counties.shape

(0, 28)

In [44]:
counties["is_simple"] = counties['boundary'].is_simple()
filtered_counties = counties[counties['is_simple'] == 0]
filtered_counties.shape

(37, 28)

Finally, you can ask yourself whether a curve is closed i.e. has a loop structure. Use the `is_closed` method for this.

In [45]:
#is_closed
sample_lines.is_closed().head()

0    0
1    0
Name: DB2GSE.ST_ISCLOSED(GEOMETRY), dtype: int64

Note: additional geospatial functionalities are available in Db2 geospatial extender (DB2GSE) and can be used through ibmdbpy with the `IdaDataBase.ida_query` function . Find out more about DB2GSE on the [IBM Knowledge Center](https://www.ibm.com/support/knowledgecenter/en/SSEPGG_11.1.0/com.ibm.db2.luw.spatial.topics.doc/doc/csbp1001.html).

__11. Operations on multiple geometry columns__

The last section of this guide shows you how to use geospatial methods which return an IdaGeoDataFrame. So far, all the methods we have showcased return an IdaGeoSeries i.e. only one column. The methods we show you here are applied on an IdaGeoDataFrame (say, `ida1`) with a second IdaGeoDataFrame as parameter (say, `ida2`). It returns a new IdaGeoDataFrame (let's call it `result`) containing three columns: IDs from `ida1`, IDs from `ida2`and the result of a given operation for each pair of input values. Let's look at an example.

In [46]:
# Distance 

counties = IdaGeoDataFrame(idadb,'SAMPLES.GEO_COUNTY',indexer='OBJECTID')
counties.set_geometry('SHAPE')
ida1 = counties[counties['NAME'] == 'Washington']
ida2 = counties[counties['NAME'] == 'Kent']

print(ida1.shape)
print(ida2.shape)

result = ida1.distance(ida2,unit = 'KILOMETER')

print(result.shape)
result.head()

(31, 19)
(5, 19)
column1_for_db2gse: SHAPE
column2_for_db2gse: SHAPE
(155, 3)


,INDEXERIDA1,INDEXERIDA2,RESULT
0,6,2231,3872.238731
1,6,1840,2280.202293
2,6,2939,3832.304173
3,6,109,2927.972900
4,6,163,4064.978945


So what happened here ? `ida1`and `ida2` are obtained with a filtering statement on the `NAME` column. There are 31 counties called Washington in the US, and 5 called Kent, hence the number of rows in each table. The two IdaGeoDataFrames have inherited their `geometry` attribute from the `counties` IdaGeoDataFrame. Their column `SHAPE` is set as geometry. `result` is a new IdaGeoDataFrame obtained by applying the `distance` method to `ida1` with `ida2` as parameter. It contains as many distances as pairs (geom1, geom2) of elements geom1 from the geometry column of `ida1`, geom2 from the geometry column of `ida2`. 31 times 5 makes 155, so we are good. 

Okay, but what exactly does this distance actually stands for? Under the hood, ibmdbpy executes an SQL query using db2gse.ST_DISTANCE. This geospatial function from Db2 returns *the shortest distance between any point in the first geometry to any point in the second geometry*, measured in the default or given units. Here we have chosen to output the result in kilometers.

A note on this particular method: our input geometries are polygons not points, so you might wonder which points are actually used to compute this distance. The output is the closest distance between both geometries, i.e. the distance between the two closest points from each polygons. 

* Methods returning a new geometry

The `difference` method and the `intersection` method respectively compute the geometry corresponding to the geospatial difference and the intersection of two input geometries. 

More precisely, `difference` returns the part of the first geometry which does not intersect with the second geometry. Both geometries must be of the same dimension. If either geometry is null, null is returned. If the first geometry is empty, an empty geometry of type ST_Point is returned. If the second geometry is empty, then the first geometry is returned unchanged.

`intersection` returns a new geometry that is the intersection of the two given geometries, represented in the spatial reference system of the first geometry. If possible, the specific type of the returned geometry will be `ST_Point`, `ST_LineString`, or `ST_Polygon`. For example, the intersection of a point and a polygon is either empty or a single point, represented as ST_MultiPoint.

In [47]:
# Difference

ida1 = counties[counties['NAME'] == 'Austin']
ida2 = counties[counties['NAME'] == 'Kent']
result = ida1.difference(ida2)
result.head()

column1_for_db2gse: SHAPE
column2_for_db2gse: SHAPE


,INDEXERIDA1,INDEXERIDA2,RESULT
0,2,109,"POLYGON ((-96.6219873342 30.0442882117, -96.61..."
1,2,163,"POLYGON ((-96.6219873342 30.0442882117, -96.61..."
2,2,2231,"POLYGON ((-96.6219873342 30.0442882117, -96.61..."
3,2,2939,"POLYGON ((-96.6219873342 30.0442882117, -96.61..."
4,2,1840,"POLYGON ((-96.6219873342 30.0442882117, -96.61..."


Note that you can apply such a method on an IdaGeoDataFrame to itself. For example here:

In [48]:
# Intersection

sample_geometries.intersection(sample_geometries).head(10)

column1_for_db2gse: GEOMETRY
column2_for_db2gse: GEOMETRY


,INDEXERIDA1,INDEXERIDA2,RESULT
0,1,1,POINT (1.000000 2.000000)
1,1,2,POINT (1.000000 2.000000)
2,1,3,POINT EMPTY
3,1,4,POINT EMPTY
4,1,5,POINT EMPTY
5,1,6,POINT EMPTY
6,1,7,POINT EMPTY
7,2,1,POINT (1.000000 2.000000)
8,2,2,"POLYGON ((0.000000 0.000000, 5.000000 0.000000..."
9,2,3,POINT EMPTY


`union` gives you the geometric union of the two input shapes. Geometry must be compatible e.g. only curves or only surfaces.

In [49]:
new_sample = sample_lines.union(sample_mlines)
new_sample.head()

column1_for_db2gse: GEOMETRY
column2_for_db2gse: GEOMETRY


,INDEXERIDA1,INDEXERIDA2,RESULT
0,1110,1110,"MULTILINESTRING ((850.000000 250.000000, 850.0..."
1,1111,1110,"MULTILINESTRING ((90.000000 90.000000, 100.000..."


Additionally, there is a bunch of methods to explore relationships between geometries, for example whether two geometries overlap (surfaces), intersect (curves), or contain, cross or touches one another. In the result column, you will obtain 0 for False, 1 for True, NaN if not applicable. Here are a few examples.

* Inclusion

`contains` indicates whether an element from the first geometry contains an element fron the second geometry.

`within` indicates whether an element from the first geometry is within an element fron the second geometry.

`equals` method can be used to look for duplications.

In [50]:
# Contains
customers = IdaGeoDataFrame(idadb,'SAMPLES.GEO_CUSTOMER',indexer='OBJECTID')
customers.set_geometry('SHAPE')

ida1 = customers[customers['INSURANCE_VALUE']>250000]
ida2 = counties[counties['NAME']=='Madison']
result = ida2.contains(ida1)

result[result['RESULT']==1].head()

column1_for_db2gse: SHAPE
column2_for_db2gse: SHAPE


,INDEXERIDA1,INDEXERIDA2,RESULT
0,134,21447,1
1,134,21450,1
2,134,21517,1
3,134,21518,1
4,134,21519,1


In [51]:
# Within
sample_geometries.within(sample_geometries).head(6)

column1_for_db2gse: GEOMETRY
column2_for_db2gse: GEOMETRY


,INDEXERIDA1,INDEXERIDA2,RESULT
0,1,3,NaN
1,1,1,1.0
2,1,4,NaN
3,1,7,0.0
4,1,5,0.0
5,1,6,0.0


In [52]:
# Reminder: how the data looks like
criterion = (sample_geometries['ID'] == 1)|(sample_geometries['ID'] == 2)|(sample_geometries['ID'] == 5)
sample_geometries[criterion].head()

,ID,GEOMETRY,min_X,min_Y,min_Z,min_M
0,1,POINT (1.000000 2.000000),1.0,2.0,None,None
1,2,"POLYGON ((0.000000 0.000000, 5.000000 0.000000...",0.0,0.0,None,None
2,5,"LINESTRING (33.000000 2.000000, 34.000000 3.00...",33.0,2.0,None,None


Comment: a point is considered to contain itself. Polygon with ID 2 contains point with ID 1. Empty geometries can't contain any object so NaN is returned. Pairs of objects without inclusion relationship get a 0.

* Intersections and contact

A handful of methods to help you explore these topics: `crosses` (whether the geometry of the first IdaGeoDataFrame crosses the second),`intersects` (whether two geometries intersect each other), `mbr_intersects` (same for minimum bounding rectangles of each geometry), `disjoint` (opposite of intersection), `overlaps` () and `touches` ().

Basic rules about geometry dimensions apply. For example, in the case of `crosses`: if the intersection of the two geometries results in a geometry that has a dimension that is one less than the maximum dimension of the two given geometries, and if the resulting geometry is not equal to any of the two given geometries, then 1 is returned. Incompatible dimensions: if the first geometry is a polygon or a multipolygon, or if the second geometry is a point or multipoint, or if any of the geometries is null value or is empty, then null is returned. Otherwise, the result is 0 (zero).

Note: `intersects` returns the exact opposite result of `disjoint`.

More details about intersections and allowed relationships between data types: [IBM Knowledge Center](https://www.ibm.com/support/knowledgecenter/en/SSEPGG_11.1.0/com.ibm.db2.luw.spatial.topics.doc/doc/rsbp4060.html).

In [53]:
# Crosses

result = tornadoes.crosses(tornadoes)
result[result["RESULT"]==1].head()

column1_for_db2gse: SHAPE
column2_for_db2gse: SHAPE


,INDEXERIDA1,INDEXERIDA2,RESULT
0,1,2700,1
1,1,5244,1
2,1,57562,1
3,8,2802,1
4,8,16886,1


In [54]:
# Intersects
result = ida1.intersects(ida2)
result.head()

column1_for_db2gse: SHAPE
column2_for_db2gse: SHAPE


,INDEXERIDA1,INDEXERIDA2,RESULT
0,3,134,0
1,3,160,0
2,3,263,0
3,3,395,0
4,3,1709,0


A last example:`overlaps`. If the intersection of the geometries results in a geometry of the same dimension but is not equal to either of the given geometries, it returns 1 . Otherwise, it returns 0.

In [55]:
# Overlaps

texas59 = tornadoes[(tornadoes["ST"]=='TX')&(tornadoes["YR"]==1959)]
texas60 = tornadoes[(tornadoes["ST"]=='TX')&(tornadoes["YR"]==1960)]
# tornado records from Texas in 1959 and 1960 respectively

result = texas60.overlaps(texas59)
result.head()

column1_for_db2gse: SHAPE
column2_for_db2gse: SHAPE


,INDEXERIDA1,INDEXERIDA2,RESULT
0,4791,4534,0
1,4791,4674,0
2,4791,4246,0
3,4791,4530,0
4,4791,4277,0


Wow! You reached the end of this notebook about ibmdbpy geospatial extension! You learned so much!

Before closing this notebook, a last step to perform: closing the connection to Db2.

___

__Close the connection__

In [56]:
idadb.close()
#idadb.reconnect()

Connection closed.


## Where to go from here?

__Congratulations!__ You are now familiar with advanced functionalities of ibmdbpy's geospatial extension! You are ready to get hands-on experience by playing with other notebooks of this series, and apply your new skills on your own data!

* Getting started with ibmdbpy :
        
    [Basics](./ibmdbpy_GettingStarted_1-basics.ipynb)
    
    
* More on ibmdbpy's geospatial extension:

    [GeoBasics](./ibmdbpy_GettingStarted_2-geo_basics.ipynb)
    

* More practice : analyze the Museums dataset, understand how to create IdaDataFrames and IdaGeoDataFrames:
        
    [Preprocessing](../MuseumsUseCase/ibmdbpy_Museums_DataAnalysis_1-preprocessing.ipynb)

    [Geospatial recommendation](../MuseumsUseCase/ibmdbpy_Museums_DataAnalysis_2-geospatial.ipynb)


* Machine learning with ibmdbpy: 
        
    [Naïve Bayes](../MachineLearning/ibmdbpy_NaiveBayes.ipynb)

    [Association Rules Mining](../MachineLearning/ibmdbpy_AssociationRulesMining.ipynb)

____

__Authors__

Eva Feillet - ML intern, IBM Cloud and Cognitive Software @ IBM Lab in Böblingen, Germany